In [ ]:
#install kaggle
!pip install -q kaggle

In [ ]:
#upload kaggle API key
from google.colab import files
files.upload()

In [ ]:
#ceate kaggle directory in user's default path
!mkdir ~/.kaggle

In [ ]:
#copy kaggle API key to kaggle directory
!cp kaggle.json ~/.kaggle/

In [ ]:
#Permission grant for the json file
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#import the kaggle breast cancer dataset
!kaggle datasets download -d paultimothymooney/breast-histopathology-images

In [ ]:
#unzip the imported kaggle breast cancer dataset
!unzip breast-histopathology-images.zip

In [ ]:
#ceate dataset directory
!mkdir /content/dataset/

In [ ]:
#ceate dataset/benign directory
!mkdir /content/dataset/benign/

In [ ]:
#ceate dataset/malignant directory
!mkdir /content/dataset/malignant/

In [ ]:
#remove the unwanted directory
!rm -rf /content/IDC_regular_ps50_idx5